#Exercise 2

In [1]:
!pip install -q pyomo

In [2]:
from pyomo.environ import * 

In [3]:
import numpy as np

\begin{align}
\max \ 35 x_1 + 57 x_2 + 48 x_3 + 20 x_4 + 15 x_5, \\
{\rm{s.t.}} \; x_1 + x_2 + x_3 & \leq 75, \\
x_3 + x_4 + x_5 & \leq 100, \\
x_1 + x_2 & \geq 10, \\ 
30000 x_1 + 95000 x_2 + 10000 x_3 + 5000 x_4 + 4000 x_5 & \leq 1500000, \\
100000 x_1 + 500000 x_2 + 200000 x_3 + 45000 x_4 + 25000 x_5 & \geq 900000, \\
x_i & \geq 0, \quad i \in \{1,2,3,4,5\}. \\
\end{align}
---

Note: $x_1, x_2, x_3, x_4, x_5$ denote the number of different types of media slots to be used for advertisement.  

In [4]:
import pandas as pd

In [5]:
data_csvfile = pd.read_csv('sample_csvfile.csv')

In [6]:
print (type(data_csvfile))

<class 'pandas.core.frame.DataFrame'>


In [7]:
data_csvfile.columns

Index(['Medium', 'Num Customers', 'Cost per advt', 'Max slots per month',
       'Reach Quality'],
      dtype='object')

In [8]:
data_csvfile['Medium']

0           Websites
1                 TV
2     DailyNewspaper
3     WeeklyMagazine
4    MonthlyMagazine
Name: Medium, dtype: object

In [9]:
data_csvfile['Num Customers']

0    100000
1    500000
2    200000
3     45000
4     25000
Name: Num Customers, dtype: int64

In [10]:
data_csvfile['Cost per advt']

0    30000
1    95000
2    10000
3     5000
4     4000
Name: Cost per advt, dtype: int64

In [11]:
data_csvfile['Max slots per month']

0    20
1    25
2    30
3     4
4     1
Name: Max slots per month, dtype: int64

In [12]:
data_csvfile['Reach Quality']

0    35
1    57
2    48
3    20
4    15
Name: Reach Quality, dtype: int64

In [13]:
data_csvfile.index

RangeIndex(start=0, stop=5, step=1)

In [14]:
len(data_csvfile.index)

5

In [15]:
# create a model
model_lab4_ex2 = ConcreteModel()

In [16]:
# we can set certain values using python variables
N = len(data_csvfile.index) #we declare a Python variable N denoting the number of decision variables  

In [17]:
print(data_csvfile)

            Medium  Num Customers  ...  Max slots per month  Reach Quality
0         Websites         100000  ...                   20             35
1               TV         500000  ...                   25             57
2   DailyNewspaper         200000  ...                   30             48
3   WeeklyMagazine          45000  ...                    4             20
4  MonthlyMagazine          25000  ...                    1             15

[5 rows x 5 columns]


In [18]:
M = 5 #we declare a Python variable M denoting the number of constraints

In [19]:
col_indices = range(N)

In [20]:
#declare the decision variables in the model
model_lab4_ex2.x = Var(col_indices)

In [21]:
model_lab4_ex2.objective = Objective(expr=summation(data_csvfile['Reach Quality'], model_lab4_ex2.x),sense=maximize)

In [22]:
model_lab4_ex2.constraints = ConstraintList()

In [23]:
#We will add the first three constraints manually
model_lab4_ex2.constraints.add(expr= model_lab4_ex2.x[0] + model_lab4_ex2.x[1] + model_lab4_ex2.x[2]<= 75  )
model_lab4_ex2.constraints.add(expr= model_lab4_ex2.x[2] + model_lab4_ex2.x[3] +  model_lab4_ex2.x[4] <= 100  )
model_lab4_ex2.constraints.add(expr= model_lab4_ex2.x[0] + model_lab4_ex2.x[1] >= 10  )

In [24]:
#Now we can add the cost constraint using the data obtained from the csv file
model_lab4_ex2.constraints.add(summation(data_csvfile['Cost per advt'], model_lab4_ex2.x) <= 1500000)


In [25]:
#Similarly  we can add the constraint about number of customers to be reached, using data obtained from the csv file
model_lab4_ex2.constraints.add(summation(data_csvfile['Num Customers'], model_lab4_ex2.x) >= 900000)


In [26]:
#Since the variables have the same upper bound and lower bound, we can simply use scalars to store them
lower_bound = 0
upper_bound = np.inf

In [27]:
#add the upper bound and lower bounds for the variables
for j in col_indices:
  model_lab4_ex2.x[j].setlb(lower_bound)
  model_lab4_ex2.x[j].setub(upper_bound)

In [28]:
model_lab4_ex2.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :   inf : False :  True :  Reals
          1 :     0 :  None :   inf : False :  True :  Reals
          2 :     0 :  None :   inf : False :  True :  Reals
          3 :     0 :  None :   inf : False :  True :  Reals
          4 :     0 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 35*x[0] + 57*x[1] + 48*x[2] + 20*x[3] + 15*x[4]

1 Constraint Declarations
    constraints : Siz

In [29]:
!apt-get install -y -qq coinor-cbc

In [30]:
opt_cbc = SolverFactory('cbc')

In [31]:
result = opt_cbc.solve(model_lab4_ex2)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [32]:
# display solution
print('\nObjective = ', model_lab4_ex2.objective())

print('\nDecision Variables')
for i in col_indices:
  print('Num slots for medium ',data_csvfile['Medium'][i], model_lab4_ex2.x[i].value)

print('\nConstraints')
model_lab4_ex2.constraints.display()


Objective =  4296.9230776

Decision Variables
Num slots for medium  Websites 4.2307692
Num slots for medium  TV 5.7692308
Num slots for medium  DailyNewspaper 65.0
Num slots for medium  WeeklyMagazine 35.0
Num slots for medium  MonthlyMagazine 0.0

Constraints
constraints : Size=5
    Key : Lower    : Body               : Upper
      1 :     None :               75.0 :      75.0
      2 :     None :              100.0 :     100.0
      3 :     10.0 :               10.0 :      None
      4 :     None : 1500000.0019999999 : 1500000.0
      5 : 900000.0 :        17882692.32 :      None


In [33]:
for i in col_indices:
  model_lab4_ex2.x[i].domain=NonNegativeIntegers

In [34]:
result = opt_cbc.solve(model_lab4_ex2)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [35]:
# display solution
print('\nObjective = ', model_lab4_ex2.objective())

print('\nDecision Variables')
for i in col_indices:
  print('Num slots for medium ',data_csvfile['Medium'][i], model_lab4_ex2.x[i].value)

print('\nConstraints')
model_lab4_ex2.constraints.display()


Objective =  4294.0

Decision Variables
Num slots for medium  Websites 7.0
Num slots for medium  TV 5.0
Num slots for medium  DailyNewspaper 63.0
Num slots for medium  WeeklyMagazine 37.0
Num slots for medium  MonthlyMagazine 0.0

Constraints
constraints : Size=5
    Key : Lower    : Body       : Upper
      1 :     None :       75.0 :      75.0
      2 :     None :      100.0 :     100.0
      3 :     10.0 :       12.0 :      None
      4 :     None :  1500000.0 : 1500000.0
      5 : 900000.0 : 17465000.0 :      None


##Ans 1 :

We are supposed to add a new constraint : $|z_1-z_2|\leq18$

where $z_1=x_1+x_2$ and $z_2=x_3+x_4+x_5$

thus, it becomes : $|(x_1+x_2)-(x_3+x_4+x_5)|\leq18$

We add the following constraints for introducing the required restrictions :

 $(x_1+x_2)-(x_3+x_4+x_5)\leq18\\
 -(x_1+x_2)+(x_3+x_4+x_5)\leq18$

 Thus, the resulting program is linear, since all our constraints and the objective function are linear.

In [36]:
model_lab4_ex2.constraints.add(model_lab4_ex2.x[0]+model_lab4_ex2.x[1]-model_lab4_ex2.x[2]-model_lab4_ex2.x[3]-model_lab4_ex2.x[4] <= 18)
model_lab4_ex2.constraints.add(-model_lab4_ex2.x[0]-model_lab4_ex2.x[1]+model_lab4_ex2.x[2]+model_lab4_ex2.x[3]+model_lab4_ex2.x[4] <= 18)

In [37]:
model_lab4_ex2.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {1, 2, 3, 4, 5, 6, 7}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   7.0 :   inf : False : False : NonNegativeIntegers
          1 :     0 :   5.0 :   inf : False : False : NonNegativeIntegers
          2 :     0 :  63.0 :   inf : False : False : NonNegativeIntegers
          3 :     0 :  37.0 :   inf : False : False : NonNegativeIntegers
          4 :     0 :   0.0 :   inf : False : False : NonNegativeIntegers

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 35*x[0] + 57*x[1] + 48*x[

In [38]:
result = opt_cbc.solve(model_lab4_ex2)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [39]:
# display solution
print('\nObjective = ', model_lab4_ex2.objective())

print('\nDecision Variables')
for i in col_indices:
  print('Num slots for medium ',data_csvfile['Medium'][i], model_lab4_ex2.x[i].value)

print('\nConstraints')
model_lab4_ex2.constraints.display()


Objective =  3378.0

Decision Variables
Num slots for medium  Websites 34.0
Num slots for medium  TV 0.0
Num slots for medium  DailyNewspaper 41.0
Num slots for medium  WeeklyMagazine 11.0
Num slots for medium  MonthlyMagazine 0.0

Constraints
constraints : Size=7
    Key : Lower    : Body       : Upper
      1 :     None :       75.0 :      75.0
      2 :     None :       52.0 :     100.0
      3 :     10.0 :       34.0 :      None
      4 :     None :  1485000.0 : 1500000.0
      5 : 900000.0 : 12095000.0 :      None
      6 :     None :      -18.0 :      18.0
      7 :     None :       18.0 :      18.0


##Ans 7 :

We are supposed to add a new constraint : $|z_3-x_3|\leq4$

where $z_3 = x_4+x_5$

thus, it becomes : $|(x_4+x_5)-x_3|\leq4$

We can introduce this by introducing two constraints :

$(x_4+x_5)-x_3\leq4\\
-(x_4+x_5)+x_3\leq4$

Thus, the resulting program is linear, since all the constraints and the objective function is linear.